In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
import re
from pathlib import Path
from functools import partial, cache, cached_property
from itertools import chain
from inspect import signature
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
import talib as ta 
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import ParameterGrid
from termcolor import colored
from joblib import Parallel, delayed
# from bhsysapi import BHSysApi

from techi import api as ti

from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs
from gioutils.blpw import BlpQuery
from gioutils.gui.pandas import millify
from gioutils.utils import (
    RepeatedTimer, 
    parse_swap_tenor_expr, 
    parse_offset,
    today, 
    get_next_n, 
    get_n_from_code,
    date_from_offset, 
    get_bday,
    get_bbg_fut_chain_ticker
)

from api.data.base import data_path, radar_path
from api.gui.params import bbg_params, spot_params, fwd_params
from api.instruments.swap import RealSwap, RollingSwap, Fly, Spread
from api.instruments.utils import parse_dates, parse_number, ez_bump
from api.instruments.strategies import FutureStrategy
from api.instruments.signals import MACDHistCrossover, ZScoreThreshold, EnsambleSignal
from api.instruments.cache import data_cache
from api.instruments.future import FutureChain, FutureSpread, FutureFly
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio
from api.instruments import config as c
from api.instruments.signals2 import BaseSignal, MACDCustomSignal, get_crossover_signal
from api.instruments.strategies2 import MACDCustomStrategy

In [3]:
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None)
c.USE_T_COST = True

In [4]:
bq = BlpQuery(timeout=50000).start()
# ba = BHSysApi(password="A&3jg5Aj")

In [5]:
notional = 1000
hist = '10y'
strategy_hist = '1y'

## Instruments Parameters

In [6]:
rates = [
    # 'ed',
    'er',
    'sfr',
    'ff',
    'sfi'
]

govt = [
    'tu',
    'fv',
    'ty',
    'us',
    'du',
    'oe',
    'rx',
    'ub',
    'ik'
]

In [7]:
rates_single_legs = range(1, 12 + 1)
govt_single_legs = range(1, 2 + 1)

fly_legs = [
    (2, 3, 4),
    (2, 4, 6),
    (2, 5, 8),
    (2, 6, 10),
    (2, 7, 12),
    (3, 4, 5),
    (3, 5, 7),
    (3, 6, 9),
    (3, 7, 11),
    (3, 8, 13),
    (4, 6, 8),
    (4, 7, 9),
    (4, 7, 11),
    (4, 8, 12),
    (5, 8, 11),
    (5, 9, 13),
    (6, 9, 12),
]

spread_legs = [
    (2, 3),
    (2, 4),
    (2, 5),
    (2, 6),
    (2, 7),
    (2, 8),
    (2, 9),
    (3, 5),
    (3, 6),
    (3, 7),
    (3, 8),
    (3, 9),
    (3, 10),
    (3, 11),
    (3, 12),
    (4, 6),
    (4, 7),
    (4, 8),
    (4, 9),
    (4, 10),
    (4, 11),
    (4, 12),
    (5, 8),
    (5, 9),
    (5, 10),
    (5, 11),
    (5, 12),
    (6, 8),
    (6, 9),
    (6, 10),
    (6, 11),
    (6, 12),
    (7, 10),
    (7, 11),
    (7, 12),
    (8, 12),
]

In [8]:
steps = dict(
    fastperiod=1,
    slowperiod=1,
    signalperiod=1,
    time_barrier=1,
    q=0.1,
    k=0.1,
    crossover_days=1,
    signal_threshold_buy=10,
    signal_threshold_sell=10,
    base_stop_periods=1,
    base_stop_mult=1,
)

In [9]:
macd_params = {
    'fastperiod': np.arange(12, 12 + steps['fastperiod'], steps['fastperiod']).tolist(),
    'slowperiod': np.arange(26, 26 + steps['slowperiod'], steps['slowperiod']).tolist(),
    'signalperiod': np.arange(9, 9 + steps['signalperiod'], steps['signalperiod']).tolist(),
}

strategy_params = {
    'time_barrier': np.arange(4, 5 + steps['time_barrier'], steps['time_barrier']).tolist(),
    'q': np.arange(0.4, 0.9 + steps['q'], steps['q']).round(int(np.abs(np.log10(steps['q'])))).tolist(),
    'k': np.arange(2, 3 + steps['k'], steps['k']).round(int(np.abs(np.log10(steps['k'])))).tolist(),
    'crossover_days': np.arange(3,  3 + steps['crossover_days'], steps['crossover_days']).tolist(),
    'base_stop_periods': np.arange(7,  7 + steps['base_stop_periods'], steps['base_stop_periods']).tolist(),
    'base_stop_mult': np.arange(2,  2 + steps['base_stop_mult'], steps['base_stop_mult']).tolist(),
}

In [10]:
macd_dynamic_params = [
    'signal_threshold_buy',
    'signal_threshold_sell',
    'slope_threshold_buy',
    'slope_threshold_sell',
]

In [11]:
macd_param_grid = ParameterGrid(macd_params)

In [12]:
signal_threshold_quantile1 = .75
signal_threshold_quantile2 = .99
slope_threshold_quantiles = [
    0.3, 
    # 0.4, 
    # 0.5
]

### Rates Single

In [13]:
def make_param_grids(
    instrument,
    macd_param_grid, 
    signal_threshold_quantile1=.75,
    signal_threshold_quantile2=.99,
    slope_threshold_quantiles=None,
    notional=1000,
    start_hist='10y',
):
    slope_threshold_quantiles = slope_threshold_quantiles or [
        0.3,
        # 0.4,
        # 0.5,
    ]
    instrument_param_grids = []
    for macd_param in macd_param_grid:
        signal = MACDCustomSignal(ti_kwargs=macd_param)
        data_ = signal.get_data(instrument)
        signal_threshold_buy = np.linspace(
            data_['macd'].quantile(1 - signal_threshold_quantile2),
            data_['macd'].quantile(1 - signal_threshold_quantile1), 
            num=steps['signal_threshold_buy']
        ).round(np.abs(round(np.log10(np.abs(data_['macd'].mean())))) + 2)
        signal_threshold_sell = np.linspace(
            data_['macd'].quantile(signal_threshold_quantile2), 
            data_['macd'].quantile(signal_threshold_quantile1), 
            num=steps['signal_threshold_sell']
        ).round(np.abs(round(np.log10(np.abs(data_['macd'].mean())))) + 2)

        slope_threshold_buy = [0, *data_.loc[data_['slope'] > 0, 'slope'].quantile(slope_threshold_quantiles).round(np.abs(round(np.log10(np.abs(data_['slope'].mean())))) + 2)]
        slope_threshold_sell = [0, *data_.loc[data_['slope'] < 0, 'slope'].quantile([*reversed(slope_threshold_quantiles)]).round(np.abs(round(np.log10(np.abs(data_['slope'].mean())))) + 2)]

        signal_dynamic_params_ = {
            'signal_threshold_buy': signal_threshold_buy,
            'signal_threshold_sell': signal_threshold_sell,
            'slope_threshold_buy': slope_threshold_buy,
            'slope_threshold_sell': slope_threshold_sell,
        }

        new_params = {
            **{k: [macd_param[k]] for k in macd_param},
            **signal_dynamic_params_,
            **strategy_params,
        }

        instrument_param_grids.append(ParameterGrid(new_params))
    return instrument_param_grids

In [14]:
rank_weight = pd.Series(
    {
        'std': -1,
        # 'var': 1,
        'cond_var': 1,
        'median': 1,
        'mean': 1,
        'sharpe_ratio': 1,
        'hit_ratio': 1,
        'drawdown': 1,
        'pnl': 1,
        'n_trades': 1
    }
)

In [15]:
def run_one(
    ticker,
    legs, 
    rank_weight,
    macd_param_grid,
    notional=1000,
    start_hist='10y',
    strategy_start_hist='1y',
    signal_threshold_quantile1=.75,
    signal_threshold_quantile2=.99,
    slope_threshold_quantiles=None,
    save_max=100, 
    stop=None,
    stop2=None,
):
    c.USE_T_COST = True
    stats = {}
    strategies = {}
    symbol = f"{ticker} {legs}"
    instrument = ff = FutureSpread.from_legs(
        ticker, 
        legs, 
        notional=notional,
        start_hist=start_hist
    )

    instrument_param_grids = make_param_grids(
        instrument=instrument,
        macd_param_grid=macd_param_grid,
        signal_threshold_quantile1=signal_threshold_quantile1,
        signal_threshold_quantile2=signal_threshold_quantile2,
        slope_threshold_quantiles=slope_threshold_quantiles,
        notional=notional,
        start_hist=start_hist,
    )
        
    stop = stop or len(instrument_param_grids)
    total = min(stop, len(instrument_param_grids)) * len(instrument_param_grids[0])
    
    for j, param in enumerate(chain.from_iterable(instrument_param_grids[:stop])):
        signal_params = {
            **{k: param[k] for k in macd_dynamic_params},
            'ti_kwargs': {k: param[k] for k in macd_params},
        }
        signal = MACDCustomSignal(**signal_params)

        custom_strategy_params = {k: param[k] for k in strategy_params}
        strategy = MACDCustomStrategy(
            instrument=instrument,
            signal=signal,
            start_hist=strategy_start_hist,
            **custom_strategy_params
        )
        strategy.run()
        if hasattr(strategy, '_summary'):
            strategies[j] = strategy
            stats[j] = strategy.stats
            if len(strategies) > save_max:
                stats_ = pd.DataFrame(stats).T
                ranks_ = (stats_.loc[:, [*rank_weight.index]] * rank_weight).rank().mean(axis=1).sort_values(ascending=False)
                to_drop = ranks_.index[-1]
                strategies.pop(to_drop)
                stats.pop(to_drop)
            print(
                f'{millify(j + 1)}/{millify(total)}', 
                symbol.upper(), 
                # {**signal_params, **custom_strategy_params}, 
                'PV',
                millify(strategy._summary['pv'].iloc[-1]),
                '    ',
                end="\r"
            )
        if j == stop2:
            break
    stats = pd.DataFrame(stats).T
    strategies = pd.Series(strategies)
    if not stats.empty:
        order = (stats.loc[:, [*rank_weight.index]] * rank_weight).rank().mean(axis=1).sort_values(ascending=False).index
        stats = stats.loc[order, :]
        strategies = strategies.loc[order]
    return stats, strategies, symbol

In [16]:
r = run_one(
    ticker='sfr',
    legs=spread_legs[0],
    macd_param_grid=macd_param_grid,
    rank_weight=rank_weight,
    stop2=9,
    save_max=1000,
)

In [17]:
res = Parallel(
    n_jobs=-1,
    verbose=100,
    # backend='multiprocessing'
)(
    delayed(run_one)(
        ticker=ticker,
        legs=legs,
        macd_param_grid=macd_param_grid,
        rank_weight=rank_weight,
        save_max=200,
        # stop2=9
    ) for ticker in rates for legs in spread_legs
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 50.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 50.5min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed: 52.4min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 52.8min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 54.3min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 55.3min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 55.5min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 57.2min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 99.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 101.1min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 101.4min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 104.3min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 105.7min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 108.0min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapse

In [21]:
ress = {r[-1]: [*r[:-1]] for r in res}

In [19]:
from gioutils.utils import to_pickle
to_pickle(ress, 'stir_spreads.pickle')
top = {k: r[1].iloc[:20] for k, r in ress.items()}
to_pickle(top, 'top_stir_spreads.pickle')

In [22]:
for key, r in ress.items():
    stats = r[0]
    # order = (stats.loc[:, [*rank_weight.index]] * rank_weight).rank().mean(axis=1).sort_values(ascending=False).index
    # stats = stats.loc[order, :]
    display(v.Html(children=key.upper(), tag='h2'))
    out = w.Output()
    with out:
        display(stats.drop_duplicates()[:10])
    display(w.HBox([out , go.FigureWidget(ress[key][1].loc[stats.index[0]]._pnl['pv'].plot())]))

Html(children=['ER (2, 3)'], tag='h2')

    'data': [{'hovertemplate': 'variable=pv<br>date=%{x}<br>value=%{y}…

Html(children=['ER (2, 4)'], tag='h2')

    'data': [{'hovertemplate': 'variable=pv<br>date=%{x}<br>value=%{y}…

Html(children=['ER (2, 5)'], tag='h2')

...

    'data': [{'hovertemplate': 'variable=pv<br>date=%{x}<br>value=%{y}…

Html(children=['SFI (7, 12)'], tag='h2')

    'data': [{'hovertemplate': 'variable=pv<br>date=%{x}<br>value=%{y}…

Html(children=['SFI (8, 12)'], tag='h2')

    'data': [{'hovertemplate': 'variable=pv<br>date=%{x}<br>value=%{y}…

In [ ]:
for key in ress:
    display(v.Html(children=key.upper(), tag='h2'))
    out = w.Output()
    with out:
        display(ress[key][0])
    display(w.HBox([out , go.FigureWidget(ress[key][1].iloc[0]._pnl['pv'].plot())]))

In [ ]:
for key in ress:
    display(v.Html(children=key.upper(), tag='h2'))
    out = w.Output()
    with out:
        display(ress[key][0])
    display(w.HBox([out , go.FigureWidget(ress[key][1].iloc[0]._pnl['pv'].plot())]))

In [35]:
all_insrtuments = {}
for k, r in ress.items():
    strategy = r[1].iloc[0]
    for instrument in strategy.instrument.instruments:
        all_insrtuments.setdefault(instrument.ticker, instrument)

In [36]:
all_insrtuments

{'er2': ER2 Future Chain 1000 contracts,
 'er3': ER3 Future Chain -2000 contracts,
 'er4': ER4 Future Chain 1000 contracts,
 'er6': ER6 Future Chain 1000 contracts,
 'er5': ER5 Future Chain -2000 contracts,
 'er8': ER8 Future Chain 1000 contracts,
 'er10': ER10 Future Chain 1000 contracts,
 'er7': ER7 Future Chain -2000 contracts,
 'er12': ER12 Future Chain 1000 contracts,
 'er9': ER9 Future Chain 1000 contracts,
 'er11': ER11 Future Chain 1000 contracts,
 'er13': ER13 Future Chain 1000 contracts,
 'sfr2': SFR2 Future Chain 1000 contracts,
 'sfr3': SFR3 Future Chain -2000 contracts,
 'sfr4': SFR4 Future Chain 1000 contracts,
 'sfr6': SFR6 Future Chain 1000 contracts,
 'sfr5': SFR5 Future Chain -2000 contracts,
 'sfr8': SFR8 Future Chain 1000 contracts,
 'sfr10': SFR10 Future Chain 1000 contracts,
 'sfr7': SFR7 Future Chain -2000 contracts,
 'sfr12': SFR12 Future Chain 1000 contracts,
 'sfr9': SFR9 Future Chain 1000 contracts,
 'sfr11': SFR11 Future Chain 1000 contracts,
 'sfr13': SFR13